En este notebook de python he diseñado un algoritmo para extraer datos del INE. En mi canal de medium, https://medium.com/@aranaz.enrique/pyhton-machine-learning-data-driven-b0e454dd508c , muestro como podemos utilizarlo y las ventajas que aporta.

In [3]:
import pandas as pd
import requests #  Librería para realizar solicitudes HTTP.
import datetime  # Librería para manejar fechas y horas.

def ine_request(ine_code):
    resultados = 999
    path_template = 'http://servicios.ine.es/wstempus/js/ES/DATOS_SERIE/{cod_serie}?nult={n_ult_datos}'
    path = path_template.format(cod_serie=ine_code, n_ult_datos=resultados)
    
    # Realizar la solicitud GET a la API
    response = requests.get(path)
    
    # Verificar si la solicitud fue exitosa
    if response.status_code != 200:
        print(f"Error: Received status code {response.status_code} for code {ine_code}")
        return None
    
    try:
        # Intentar decodificar la respuesta JSON
        json_request = response.json()
    except ValueError:
        print(f"Error: Unable to decode JSON for code {ine_code}")
        return None
    
    return json_request

# Creamos una lista con un código del INE (ine_code) y una descripción (descripcion) asociada a cada código.
codigos = [
    {"ine_code": "ECP320", "descripcion": "Población Total Nacional"},
    {"ine_code": "ECP701", "descripcion": "Población Extranjera"}
]

In [5]:
#Inicialización de Listas
anyo_lista = list()
mes_lista = list()
dia_lista = list()
fecha_completa_lista = list()
variable_lista = list()
valor_lista = list()
descripcion_lista = list()

# Bucle para Obtener Datos

for codigo in codigos:
    datos = ine_request(codigo['ine_code']) #  Llama a la función ine_request para obtener los datos de la API del INE para el código actual.
    
    if datos is None: # Si datos es None (lo que significa que hubo un error en la solicitud), el bucle continúa con el siguiente código.
        continue
    
    nombre_variable = datos['Nombre']
    descripcion = codigo['descripcion']
    for dato in datos['Data']:
        fecha = datetime.date.fromtimestamp(dato['Fecha'] // 1000)
        anyo = fecha.year
        mes = fecha.month
        dia = fecha.day
        valor = dato['Valor']
        fecha_completa = fecha.strftime('%Y-%m-%d')

        anyo_lista.append(anyo)
        mes_lista.append(mes)
        dia_lista.append(dia)
        fecha_completa_lista.append(fecha_completa)
        variable_lista.append(nombre_variable)
        valor_lista.append(valor)
        descripcion_lista.append(descripcion)

# Creación del DataFrame
df = pd.DataFrame({
    'Año': anyo_lista,
    'Mes': mes_lista,
    'Día': dia_lista,
    'Fecha Completa': fecha_completa_lista,
    'Dato': variable_lista,
    'Descripcion': descripcion_lista,
    'Valor': valor_lista
})

df


,Año,Mes,Día,Fecha Completa,Dato,Descripcion,Valor
0,1971,1,1,1971-01-01,Total Nacional. Todas las edades. Total. Pobla...,Población Total Nacional,34040642.0
1,1971,7,1,1971-07-01,Total Nacional. Todas las edades. Total. Pobla...,Población Total Nacional,34216856.0
2,1972,1,1,1972-01-01,Total Nacional. Todas las edades. Total. Pobla...,Población Total Nacional,34408338.0
3,1972,7,1,1972-07-01,Total Nacional. Todas las edades. Total. Pobla...,Población Total Nacional,34595886.0
4,1973,1,1,1973-01-01,Total Nacional. Todas las edades. Total. Pobla...,Población Total Nacional,34800600.0
...,...,...,...,...,...,...,...
163,2023,7,1,2023-07-01,Total Nacional. Extranjero. Todas las edades. ...,Población Extranjera,6338413.0
164,2023,10,1,2023-10-01,Total Nacional. Extranjero. Todas las edades. ...,Población Extranjera,6425134.0
165,2024,1,1,2024-01-01,Total Nacional. Extranjero. Todas las edades. ...,Población Extranjera,6500332.0
166,2024,4,1,2024-04-01,Total Nacional. Extranjero. Todas las edades. ...,Población Extranjera,6586936.0
